---
title: "Agents - Poole & Mackworth"
format: html
page-layout: full
code-line-numbers: true
code-block-border: true
toc: true
toc-location: left
number-sections: true
jupyter: python3
---

  - Readings: Chapters 1 and 2

# Representing Agents and Environments

In [1]:
import random, math, time
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline


In [2]:
# A simple way to trace the intermediate steps of algorithms

class Displayable(object):
    """Class that uses 'display'.
    The amount of detail is controlled by max_display_level
    """
    max_display_level = 1   # can be overridden in subclasses or instances

    def display(self,level,*args,**nargs):
        """print the arguments if level is less than or equal to the
        current max_display_level.
        level is an integer.
        the other arguments are whatever arguments print can take.
        """
        if level <= self.max_display_level:
            print(*args, **nargs) 

# Agents and Environment

![](https://artint.info/3e/html/x6.png)

- An agent takes the precept, updates its internal state, and output its next action.
- An agent implements the method *select_action* that takes percept and returns its next action.

In [ ]:
class Agent(Displayable):

    def initial_action(self, percept):
        """return the initial action."""
        return self.select_action(percept)   # same as select_action

    def select_action(self, percept):
        """return the next action (and update internal state) given percept
        percept is variable:value dictionary
        """
        raise NotImplementedError("go")   # abstract method

- An environment takes in actions of the agents, updates its internal state and returns the next percept, using the method *do*.

In [ ]:
class Environment(Displayable):
    
    def initial_percept(self):
        """returns the initial percept for the agent"""
        raise NotImplementedError("initial_percept")   # abstract method

    def do(self, action):
        """does the action in the environment
        returns the next percept """
        raise NotImplementedError("Environment.do")   # abstract method

- The simulator lets the agent and the environment take turns in updating their states and returning the action and the percept.

In [ ]:
class Simulate(Displayable):
    """simulate the interaction between the agent and the environment
    for n time steps.
    Returns a pair of the agent state and the environment state.
    """
    def __init__(self,agent, environment):
        self.agent = agent
        self.env = environment
        self.percept = self.env.initial_percept()
        self.percept_history = [self.percept]
        self.action_history = []
        
    def go(self, n):
        for i in range(n):
            action = self.agent.select_action(self.percept)
            self.display(2,f"i={i} action={action}")
            self.percept = self.env.do(action)
            self.display(2,f"      percept={self.percept}")

# Case Study - Paper buying agent and Environment

 - The environment state is given in terms of the time and the amount of paper in stock.
 - It also remembers the in-stock history and the price history.
 - The percept consists of the price and the amount of paper in stock.
 - The action of the agent is the number to buy.

In [ ]:
class TP_env(Environment):
    
    price_delta = [0, 0, 0, 21, 0, 20, 0, -64, 0, 0, 23, 0, 0, 0, -35,
        0, 76, 0, -41, 0, 0, 0, 21, 0, 5, 0, 5, 0, 0, 0, 5, 0, -15, 0, 5,
       0, 5, 0, -115, 0, 115, 0, 5, 0, -15, 0, 5, 0, 5, 0, 0, 0, 5, 0,
       -59, 0, 44, 0, 5, 0, 5, 0, 0, 0, 5, 0, -65, 50, 0, 5, 0, 5, 0, 0,
       0, 5, 0] 
    
    sd = 5  # noise standard deviation

    def __init__(self):
        """paper buying agent"""
        self.time=0
        self.stock=10
        self.stock_history = []  # memory of the stock history
        self.price_history = []  # memory of the price history

    def initial_percept(self):
        """return initial percept"""
        
        self.stock_history.append(self.stock)
        
        self.price = round(234 + self.sd*random.gauss(0,1))
        self.price_history.append(self.price)
        
        return {'price': self.price,
                'instock': self.stock}

    def do(self, action):
        """does action (buy) and returns percept consisting of price and instock"""
      
        used = select_from_dist({6:0.1, 5:0.1, 4:0.1, 3:0.3, 2:0.2, 1:0.2})
        # used = select_from_dist({7:0.1, 6:0.2, 5:0.2, 4:0.3, 3:0.1, 2:0.1}) # uses more paper
        
        bought = action['buy']
        
        self.stock = self.stock + bought - used
        self.stock_history.append(self.stock)
        
        self.time += 1
        self.price =  round(self.price
                        + self.price_delta[self.time%len(self.price_delta)] # repeating pattern
                        + self.sd*random.gauss(0,1)) # plus randomness
        
        self.price_history.append(self.price)
        
        return {'price': self.price,
                'instock': self.stock}

In [ ]:
class TP_agent(Agent):
    
    def __init__(self, initial_percept):
        self.spent = 0
        self.ave = self.last_price = initial_percept['price']
        self.instock = initial_percept['instock']
        self.buy_history = []
        
    def select_action(self, percept):
        """return next action to carry out
        """
        self.last_price = percept['price']
        self.ave = self.ave + (self.last_price - self.ave) * 0.05 # 20 day rolling average
        
        self.instock = percept['instock']
        
        if self.last_price < 0.9 * self.ave and self.instock < 60:
            tobuy = 48
        elif self.instock < 12:
            tobuy = 12
        else:
            tobuy = 0
        
        self.spent += tobuy*self.last_price
        self.buy_history.append(tobuy)
        
        return {'buy': tobuy}

## Rolling Average Reference:
  - https://artint.info/3e/html/ArtInt3e.A1.S1.html#A1.E1

In [ ]:
def select_from_dist(item_prob_dist):
    """ returns a value from a distribution.
    item_prob_dist is an item:probability dictionary, where the
        probabilities sum to 1.
    returns an item chosen in proportion to its probability
    """
    ranreal = random.random()
    for (it,prob) in item_prob_dist.items():
        if ranreal < prob:
            return it
        else:
            ranreal -= prob
    raise RuntimeError(f"{item_prob_dist} is not a probability distribution")

In [ ]:
random.seed(1)

Simulate.max_display_level = 1

env = TP_env()
ag = TP_agent(env.initial_percept())
sim = Simulate(ag,env)

sim.go(10)

In [ ]:
class Plot_history(object):
    """Set up the plot for history of price and number in stock"""
    def __init__(self, ag, env):
        self.ag = ag
        self.env = env
        plt.ion()
        plt.xlabel("Time")
        plt.ylabel("Value")
        

    def plot_env_hist(self):
        """plot history of price and instock"""
        num = len(env.stock_history)
        plt.plot(range(num),env.price_history,label="Price")
        plt.plot(range(num),env.stock_history,label="In stock")
        plt.legend()
        plt.draw()

    def plot_agent_hist(self):
        """plot history of buying"""
        num = len(ag.buy_history)
        plt.bar(range(1,num+1), ag.buy_history, label="Bought")
        plt.legend()
        plt.draw()

In [ ]:
pl = Plot_history(ag,env); pl.plot_env_hist(); pl.plot_agent_hist()

## Sampling from the given probability distribution

In [ ]:
#random.seed(123)

d = {'a': 0.1, 'b': 0.2, 'c': 0.3, 'd': 0.4}
counts = {'a': 0, 'b': 0, 'c': 0, 'd': 0}

for i in range(10000):
    choice = select_from_dist(d)
    counts[choice] += 1

print(counts)
    

# Case Study - Hierarchical Controller (Optional)

![](https://artint.info/3e/html/x3.png)

## Delivery Robot
   ![](https://artint.info/3e/html/x10.png)
  - Required to visit a sequence of named locations while avoiding the obstacles
  -  The robot is given a high-level plan consisting of a list of named locations to visit in sequence.
  -  The robot needs to sense the world and to move in the world in order to carry out the plan.
    

## Environment

- The environment defines the walls.


In [ ]:
trace_points = []

class Rob_env(Environment):
    def __init__(self, walls = {}):
        """walls is a set of line segments 
               where each line segment is of the form ((x0,y0),(x1,y1))
        """
        self.walls = walls

## Agent Body
 - Defines the agent body

In [ ]:
class Rob_body(Environment):
    def __init__(self, env, init_pos=(0,0,90)):
        """ env is the current environment
        init_pos is a triple of (x-position, y-position, direction) 
            direction is in degrees; 0 is to right, 90 is straight-up, etc
        """
        self.env = env
        self.rob_x, self.rob_y, self.rob_dir = init_pos
        self.turning_angle = 18   # degrees that a left makes
        self.whisker_length = 6   # length of the whisker
        self.whisker_angle = 30   # angle of whisker relative to robot
        self.crashed = False
        
        # The following are data structures maintained:
        self.history = [(self.rob_x, self.rob_y)] # history of (x,y) positions
        self.wall_history = []     # history of hitting the wall

    def percept(self):
        return {'rob_x_pos':self.rob_x, 'rob_y_pos':self.rob_y,
                'rob_dir':self.rob_dir, 'whisker':self.whisker(), 'crashed':self.crashed}
    
    initial_percept = percept  # use percept function for initial percept too

    def do(self,action):
        """ action is {'steer':direction}
        direction is 'left', 'right' or 'straight'
        """
        if self.crashed:
            return self.percept()

        global trace_points
        
        direction = action['steer']  
        compass_deriv = {'left':1,'straight':0,'right':-1}[direction]*self.turning_angle
        self.rob_dir = (self.rob_dir + compass_deriv +360)%360  # make in range [0,360)
        rob_x_new = self.rob_x + math.cos(self.rob_dir*math.pi/180)
        rob_y_new = self.rob_y + math.sin(self.rob_dir*math.pi/180)
        path = ((self.rob_x,self.rob_y),(rob_x_new,rob_y_new))
        
        if any(line_segments_intersect(path,wall) for wall in self.env.walls):
            self.crashed = True
            trace_points.append((self.rob_x, self.rob_y, "r*"))
        
        self.rob_x, self.rob_y = rob_x_new, rob_y_new 
        self.history.append((self.rob_x, self.rob_y))
        
        if not self.crashed:
            trace_points.append((self.rob_x, self.rob_y, "go"))
        
        return self.percept()

    def whisker(self):
        """returns true whenever the whisker sensor intersects with a wall
        """
        whisk_ang_world = (self.rob_dir-self.whisker_angle)*math.pi/180
            # angle in radians in world coordinates
        wx = self.rob_x + self.whisker_length * math.cos(whisk_ang_world)
        wy = self.rob_y + self.whisker_length * math.sin(whisk_ang_world)
        whisker_line = ((self.rob_x,self.rob_y),(wx,wy))
        hit = any(line_segments_intersect(whisker_line,wall)
                    for wall in self.env.walls)
        if hit:
            self.wall_history.append((self.rob_x, self.rob_y))
            trace_points.append((self.rob_x, self.rob_y, "ro"))
                
        return hit

In [ ]:
def line_segments_intersect(linea,lineb):
    """returns true if the line segments, linea and lineb intersect.
    A line segment is represented as a pair of points.
    A point is represented as a (x,y) pair.
    """
    ((x0a,y0a),(x1a,y1a)) = linea
    ((x0b,y0b),(x1b,y1b)) = lineb
    da, db = x1a-x0a, x1b-x0b
    ea, eb = y1a-y0a, y1b-y0b
    
    denom = db * ea - eb * da
    
    if denom==0:    # line segments are parallel
        return False
    
    cb = (da*(y0b-y0a)-ea*(x0b-x0a))/denom  # position along line b
    if cb<0 or cb>1:
        return False
    
    ca = (db*(y0b-y0a)-eb*(x0b-x0a))/denom # position along line a
    return 0<=ca<=1

## Middle Layer
 - The middle layer acts like both a controller (for the environment layer) and an environment for the upper layer.
 - It has to tell the environment how to steer. Thus it calls env.do(...)
 - It also is told the position to go to and the timeout. Thus it also has to implement do(...)

In [ ]:
class Rob_middle_layer(Environment):
    def __init__(self,env):
        self.env=env
        self.percept = env.initial_percept()
        self.straight_angle = 11 # angle that is close enough to straight ahead
        self.close_threshold = 2  # distance that is close enough to arrived
        self.close_threshold_squared = self.close_threshold**2 # just compute it once

    def initial_percept(self):
        return {}

    def do(self, action):
        """action is {'go_to':target_pos,'timeout':timeout}
        target_pos is (x,y) pair
        timeout is the number of steps to try
        returns {'arrived':True} when arrived is true
             or {'arrived':False} if it reached the timeout 
        """
        if 'timeout' in action:
            remaining = action['timeout']
        else:
            remaining = -1    # will never reach 0
            
        target_pos = action['go_to']
        arrived = self.close_enough(target_pos)
        
        while not arrived and remaining != 0:
            self.percept = self.env.do({"steer":self.steer(target_pos)})
            remaining -= 1
            arrived = self.close_enough(target_pos)
        
        return {'arrived':arrived}

    def steer(self,target_pos):
        """The following method determines how to steer depending on whether the goal
        is to the right or the left of where the robot is facing.
        """
        if self.percept['whisker']:
            self.display(3,'whisker on', self.percept)
            return "left"
        else:
            return self.head_towards(target_pos)

    def head_towards(self,target_pos):
            """ given a target position, return the action that heads towards that position
            """
            gx,gy = target_pos
            rx,ry = self.percept['rob_x_pos'],self.percept['rob_y_pos']
            goal_dir = math.acos((gx-rx)/math.sqrt((gx-rx)*(gx-rx)
                                                   +(gy-ry)*(gy-ry)))*180/math.pi
            if ry>gy:
                goal_dir = -goal_dir
            goal_from_rob = (goal_dir - self.percept['rob_dir']+540)%360-180
            assert -180 < goal_from_rob <= 180
            if goal_from_rob > self.straight_angle:
                return "left"
            elif goal_from_rob < -self.straight_angle:
                return "right"
            else:
                return "straight"

    def close_enough(self,target_pos):
        gx,gy = target_pos
        rx,ry = self.percept['rob_x_pos'],self.percept['rob_y_pos']
        return (gx-rx)**2 + (gy-ry)**2 <= self.close_threshold_squared

## Top Layer
 - The top layer treats the middle layer as its environment.
 - Note that the top layer is an environment for us to tell it what to visit.

In [ ]:
class Rob_top_layer(Environment):
    def __init__(self, middle, timeout=200, locations = {'mail':(-5,10), 
                          'o103':(50,10), 'o109':(100,10),'storage':(101,51)} ):
        """middle is the middle layer
        timeout is the number of steps the middle layer goes before giving up
        locations is a loc:pos dictionary 
            where loc is a named location, and pos is an (x,y) position.
        """
        self.middle = middle
        self.timeout = timeout  # number of steps before the middle layer should give up
        self.locations = locations
        
    def do(self,plan):
        """carry out actions.
        actions is of the form {'visit':list_of_locations}
        It visits the locations in turn.
        """
        to_do = plan['visit']
        for loc in to_do:
            position = self.locations[loc]
            arrived = self.middle.do({'go_to':position, 'timeout':self.timeout})
            self.display(1,"Arrived at",loc,arrived)

## Plotting

In [ ]:
class Plot_env(Displayable):
    def __init__(self, body,top):
        """sets up the plot
        """
        self.body = body
        self.top = top
        self.redraw()

    def redraw(self):

        opacity = 0.5
        for i in range(len(trace_points)):
            clear_output(wait = True)
            fig = plt.figure(figsize=(8,8))
            plt.axes().set_aspect('equal')
        
            for wall in self.body.env.walls:
                ((x0,y0),(x1,y1)) = wall
                plt.plot([x0,x1],[y0,y1],"-k",linewidth=3)
            for loc in self.top.locations:
                (x,y) = self.top.locations[loc]
                plt.plot([x],[y],"k<")
                plt.text(x+1.0,y+0.5,loc) # print the label above and to the right
            
            for point in trace_points[:i+1]:
                if point[2] == 'r*':
                    plt.plot([point[0]],[point[1]],point[2], markersize=20)    
                else:
                    plt.plot([point[0]],[point[1]],point[2], alpha=opacity)
            
            plt.show()
            #time.sleep(0.0001)
            



In [ ]:
env = Rob_env({((20,0),(30,20)), ((70,-5),(70,25))})
body = Rob_body(env)
middle = Rob_middle_layer(body)
top = Rob_top_layer(middle)
trace_points = []

In [ ]:
top.do({'visit':['o109','storage','o109','o103', 'mail']})

In [ ]:
trace_points[:10]

In [ ]:
Plot_env(body,top);

In [ ]:
len(trace_points)

In [ ]:
trap_env = Rob_env({((10,-21),(10,0)), ((10,10),(10,31)), ((30,-10),(30,0)),
                    ((30,10),(30,20)),  ((50,-21),(50,31)), ((10,-21),(50,-21)),
                    ((10,0),(30,0)),  ((10,10),(30,10)),  ((10,31),(50,31))})
trap_body = Rob_body(trap_env,init_pos=(20,-25,90))
trap_middle = Rob_middle_layer(trap_body)
trap_top = Rob_top_layer(trap_middle,locations={'goal':(71,0)})
trace_points = []

Rob_body.max_display_level = 5

In [ ]:
trap_top.do({'visit':['goal']})

In [ ]:
Plot_env(trap_body,trap_top);